In [1]:
from dotenv import dotenv_values
import os

config = dotenv_values(".env.secret")  # load sensitive variables
print(config.keys())

for c, v in config.items():
    os.environ[c] = v

odict_keys(['WANDB_API_KEY', 'WANDB_NAME', 'WANDB_NOTES', 'WANDB_NOTEBOOK_NAME'])


In [2]:
import wandb

wandb_key = os.environ["WANDB_API_KEY"]
wandb.login(key=wandb_key)

wandb: Currently logged in as: wilber-quito (deepsat). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch

torch.cuda.is_available()

True

In [4]:
import torch

torch.cuda.device_count()

1

In [5]:
import torch

torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

In [6]:
import os

max(1, os.cpu_count() -1)

11

In [7]:
from src.data import HousePricingDataModule

data_module = HousePricingDataModule()
data_module.prepare_data()

for stage in ["fit"]:
    data_module.setup(stage=stage)

[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)
[INFO]: Setting up fit dataloader


In [8]:
import os
import torch
from lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
import wandb
from src.model import HousePricingModel

project_name = os.environ["WANDB_NAME"]
wandb_logger = WandbLogger(project=project_name)

train_dataloader = data_module.train_dataloader()
val_dataloader = data_module.val_dataloader()

in_features = data_module.in_features()
model = HousePricingModel(in_features)

max_epochs = 1000

H = {
    "accelerator": "gpu" if torch.cuda.is_available() else "cpu",
    "in_features": in_features,
    "max_epochs": max_epochs
}

for c, v in H.items():
    wandb_logger.experiment.config[c] = v

trainer = Trainer(accelerator=H["accelerator"], max_epochs=H["max_epochs"], logger=wandb_logger)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 541 K 
------------------------------------
541 K     Trainable params
0         Non-trainable params
541 K     Total params
2.168     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (31) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 999: 100%|██████████| 31/31 [00:00<00:00, 104.87it/s, v_num=lr72]

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 31/31 [00:00<00:00, 99.90it/s, v_num=lr72] 


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,▃▆▂▂▃▃▄▂▁▃▁▄▃▂▁▁▄▂▁▆▁▆▃▁▁▁█▁▁▂▂▂▂▂▁▂▂▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▂▁▃▁▂▁▂▃▁▁▂▁▂▂▁▂▂▂▂▂▂▁▁▁▂▁▂▁▂▂▃▁▂▂▂▁▂▃▂
epoch,999
train_loss,69999456.0
trainer/global_step,30999
val_loss,783426368.0
